# Optimisation - Groupe 1

## Charge d'une flotte de véhicules électriques

On s’intéresse dans ce sujet à la charge de plusieurs véhicules électriques connectés à une station de recharge. Chaque véhicule est décrit par une heure et un état de charge d’arrivée et l’utilisateur indique au système d’ordonnancement une heure et un état de charge de récupération. On cherche dans ce contexte à assurer ces demandes, tout en minimisant le coût électrique de recharge et en assurant que la puissance électrique fournie aux différents véhicules ne dépasse pas la puis- sance maximale fournie par le réseau. On s’appuiera sur des données disponibles sur Oasis.

### 1. Modélisation

1. Formuler la fonction objectif du problème. On pourra supposer l’existence de deux tarifs électriques (heures pleines et heures creuses).
2. Formuler un modèle dynamique reliant l’état de charge d’un véhicule et la puissance qui lui est fournie.
3. Proposer une méthodologie d’identification de ce modèle basée sur les données fournies sur Oasis et la mettre en oeuvre.
4. Formaliser les contraintes restantes du problème et formuler le problème de minimisation correspondant. On précisera en particulier les variables de décision, les contraintes (au nombre de six) et la fonction objectif du problème.

### 2. Etude et résolution numérique

On s’intéresse dans un premier temps au cas d’un seul véhicule.
1. Etudier le problème (convexité, conditionnement, existence et unicité d’une solution) par la méthode, analytique ou numérique, de votre choix. Proposer une méthode de résolution adaptée aux caractéristiques du problème.
2. Proposer des ordres de grandeurs pour les variables du problème et développer un algorithme de résolution. Commenter les résultats obtenus.

### 3. Etude avancée


On se propose maintenant d’étudier le problème de plusieurs véhicules.
3. Etudier la faisabilité d’une méthode de décomposition-coordination (présentée en annexe).
4. Appliquer cette méthodologie dans le cas de deux véhicules. L’implémenter. Commenter les résultats obtenus.

#### Annexe : Algorithme de décomposition/coordination
L’algorithme de décomposition/coordination par méthode d’Uzawa permet de résoudre un problème d’optimisation dans le cas où :
- le coût $f$ se décompose comme une somme de différents termes ne dépendant que d’une variable de décision
$$ f (x) = f_1(x_1) + . . . + f_n(x_n) $$
- les containtes inégalités (et égalités) $c$ se décomposent comme une somme de différents termes ne dépendant que d’une variable de décision
$$ c(x) = c_1(x_1) + . . . + c_n(x_n)$$
Le principe de l’algorithme est alors de "décomposer" le problème à l’aide du Lagrangien, de résoudre n problèmes d’optimisation sous contraintes à l’aide des multiplicateurs de Lagrange et de "coordonner" les solutions pour mettre à jour les variables couplées (les multiplicateurs).

Il prend la forme suivante :

##### Algorithme 1 (Décomposition/coordination)
1. Initialisation ($k = 0$) : on choisit le multiplicateur $\lambda_0$ et le pas $\rho$.
2. Décomposition : on résout les $n$ problèmes :
$$ min (f_i(x_i) + \lambda^k c_i(x_i)) $$
dont on note $x^k_i$ les solutions.
3. Coordination : effectuer les mises à jour des multiplicateurs
$$ \lambda^{k+1} =P(\lambda^k +\rho [c_1(x^k_1)+...+c_n(x^k_n)]) $$
où $P$ est la projection sur $\mathbb{(R^+)}^m$
4. Test d’arrêt : si $|\lambda^{k+1} − \lambda^k|$ est suffisamment petit, on arrête l’algorithme. Sinon, $k = k + 1$ et on revient à l’étape 2.

### Propositions d'Antoine :

# Exploration du problème

### Fonction objectif

On est face à un problème de répartition de puissance : on peut représenter le problème par un vecteur colonne de $N$ éléments représentant chacun une voiture et dont la composante $i$ représente la fraction de puissance du centre $P$ allouée à la voiture sur la place $i$. En notant ce vecteur $v$ on aura alors : 
    $\sum \limits_{i=1}^N v_i \le 1$ et 
    $\forall i \in [1,N], v_i \in [0,1]$. Ces $(v_i)_{i \in [1;N]}$ sont nos variables de décisions.

Etant donné ce vecteur $v(t)$ on peut alors donner le coût total de charge : $f(v) = \sum\limits_{i=1}^N \displaystyle \int_{t_{i_0}}^{t_{i_1}} v_i(t)*c(t) \, \mathrm{d}t := \sum\limits_{i=1}^N f_i(v)$

En notant $ c : t \in \mathbb R^+ \Longrightarrow c(t) $ la fonction de coût de la puissance électrique en fonction du temps en seconde(la référence de temps est prise à EPOCH, le $1^{er}$ janvier 1970 à 00h), prenant en compte le tarif des heures pleines et heures creuses :
$$ t \in [6;22], c(t) = c_p$$
$$ t \in [0;6]\cup[22;24], c(t) = c_c$$
En ayant converti le temps en heures de la manière suivante : $$\lfloor t/3600 \rfloor \pmod {24}$$

Enfin on note $P$ la puissance totale du centre, ainsi que $batt_i(t) =  batt_i(t_{i_0}) + \displaystyle \int_{t_{i_0}}^{t} rech(P*v_i(x), batt_i(x)) \, \mathrm{d}x  $ la fonction représentant la batterie de la voiture $i$ à l'instant $t$, où l'on aura noté $rech$ la fonction de recharge d'une batterie, qui prend donc en argument la puissance reçue et la charge de la batterie

### Modèle de recharge en fonction de la puissance reçue

J'ai un peu cherché et j'ai demandé conseil à notre chargé de TD, on va voir ce qu'il répond

Parce que le problème est que beaucoup de modèles sont possibles, notamment en fonction de la batterie du véhicule... On a assez peu d'infos là dessus et ce choix va bcp conditionner nos solutions

### Contraintes

Là dessus on a assez peu de choix (on est assez contraints lol). Le sujet parle de 6 contraintes, j'ai trouvé celles-ci :
 - Puissance max autorisée : $\forall t,  \sum \limits_{i=1}^N v_i(t) \leq 1$
 - Heure d'arrivée de la voiture $i$ : $t_{i_0} $. Contrainte associée : $v_i(t_i)_{t_i \leq t_{i_0}} = 0$ 
 - Heure de départ de la voiture $i$ : $t_{i_1} $. Contrainte associée : $v_i(t_i)_{t_i \ge t_{i_1}} = 0$ 
 - Taux de charge initial : $c_{i_0}$. Contrainte associée : $batt_i(t_{i_0}) = c_{i_0}$ 
 - Taux de charge final : $c_{i_1}$. Contrainte associée : $batt_i(t_{i_1}) = c_{i_1}$ 
 - Modèle de recharge de la batterie.
 
 Ce sont les seules contraintes qui agissent, selon moi, sur les variables de décisions. Pour les 5 premières l'influence sur le vecteur $v$ me semble évidente, et pour la dernière la modification du modèle de recharge va changer la courbe de recharge des batteries et influencer nos solutions.
 
 Nous pourrons d'ailleurs changer ce modèle afin d'observer son influence sur nos solutions
 

## Problème de minimisation

On fait donc face au problème suivant :

$\min\limits_{\forall t,  \sum \limits_{i=1}^N v_i(t) \leq 1 , v_i(t_i)_{t_i \leq t_{i_0}} = 0 , v_i(t_i)_{t_i \ge t_{i_1}} = 0 , c_{i_0} + \displaystyle \int_{t_{i_0}}^{t_{i_1}} rech(P*v_i(x), batt_i(x)) \, \mathrm{d}x = c_{i_1}}
( \sum\limits_{i=1}^N \displaystyle \int_{t_{i_0}}^{t_{i_1}} P*v_i(t)*c(t) \, \mathrm{d}t )
$

Ou plus succintement :

$\min\limits_{\forall t,  \sum \limits_{i=1}^N v_i(t) \leq 1 , v_i(t_i)_{t_i \leq t_{i_0}, t_i \ge t_{i_1}} = 0 , v_i(t_i)_{t_i \ge t_{i_1}} = 0 , batt_i(t_{i_0}) = c_{i_0}, batt_i(t_{i_1}) = c_{i_1}}
( f(v) )
$

# Seconde modélisation

## Cadre d'étude

On discrétise le temps en T points : $(t_i)_{i \in [1,T]}$. Les voitures sont au nombre de N $(voitures_i)_{i \in [1,N]}$.

On se munit également d'un vecteur ligne de coût de l'électricité : $ C := (c_i)_{i \in [1,T]}$ et d'une matrice $V$ dont l'élément d'indice $(i,j)$ représente la fraction de puissance totale allouée à la voiture $j$ au temps $t_i$

On notera $i_0$ et $i_1$ les temps d'arrivée et de départ de la voiture $i$, ainsi que $Charge_i^0$ et $Charge_i^1$ ses taux de charge à l'arrivée et au départ du centre.

La batterie de la voiture $j$ au temps $i$ sera représentée par l'élément d'indice $(i,j)$  de la matrice $Charge$.

On note enfin $U$ la tension nominale de charge, $P$ la puissance totale du centre.

## Formalisation du problème

### Fonction objectif

La fonction a minimser est la somme des coûts de la puissance réquisitionnée par le centre.

Nous pouvons la représenter par : $f(V) = \sum\limits_{i=1}^N \displaystyle \int_{0}^{T} c(t)*V_i(t) \, \mathrm{d}t = \sum\limits_{i = 1, j = 1}^{i = N, j = T} c_i*v_{i,j} = sum(c*V) = \sum\limits_{i=1}^N f_i(V)$

On on a noté $f_i$ la fonction qui à la matrice $V$ associe le coût de la voiture $i$ sur tout l'intervalle de temps : $f_i(V) = c*V^T[i]$

### Contraintes

#### Contraintes pré-analyse

Nous avons tout d'abord identifié 6 contraintes dans ce problème, les voici :

 - Puissance max autorisée : $\forall i \in [0,T],  \sum \limits_{j=1}^N v_{i,j} \leq 1$
 - Heure d'arrivée de la voiture $j$ : $t_{j_0} $. Contrainte associée : $v_{{i,j}_{i \leq i_0}} = 0$ 
 - Heure de départ de la voiture $j$ : $t_{j_1} $. Contrainte associée : $v_{{i,j}_{i \ge i_1}} = 0$ 
 - Taux de charge initial. Contrainte associée : $Charge_{i_0,i} = Charge_i^0$ 
 - Taux de charge final. Contrainte associée : $Charge_{i_1,i} = Charge_i^1$ 
 - Modèlisation de la charge de la batterie

#### Modélisation de la batterie

Nous avons reçu un échantillon de données représentant le taux de charge d'une batterie et l'intensité de chargement en fonction du temps. A partir de ces données nous avons pu lier la vitesse de charge d'une batterie en fonction de l'intensité de chargement.

![donnees](figures/affichage_donnees.png)

On repère ici différents palliers d'intensité de charge. Ceci nous a incité à créer le graphique suivant

![donnees](figures/affichage_donnees_2.png)

On voit ici se dessiner un modèle linéaire, très bruité. Nous avons donc décidé de faire la moyenne de la vitesse de charge pour chaque pallier d'intensité (pour réduire ce bruit), puis d'appliquer la méthode des moindres carrés pour trouver la droite modélisant le mieux cet échantillon de données

![modele](figures/modele_de_recharge_lstsq_test.png)

On obtient donc la relation suivante :

$Charge_{i+1,j} = Charge_{i,j} + \Delta t * (a * P *\frac{V_{i,j}}{U} + b)$ avec $a$ le coefficient directeur et $b$ l'ordonnée à l'origine du modèle linéaire. 

On a donc : $Charge_{i_1,i} - Charge_{i_0,i} = \Delta t * P * a * \frac{\sum\limits_{i = i_0}^{i_1} V_{i,j}}{U} + (i_1 - i_0) * b * \Delta t$ 

Or on a trouvé b de l'ordre de $10^{-3}$ donc négligable ici et on a la nullité de $V_{i,j}$ en dehors des termes de la somme. Ceci est cohérent avec le modèle choisi : si l'intensité est nulle, la voiture au repos voit sa batterie rester constante.

On a donc enfin : $Charge_{i_1,i} - Charge_{i_0,i} = \Delta t * P * a * \frac{\sum\limits_{i = 0}^{T} V_{i,j}}{U}$

D'où la contrainte simplifiée suivante : $\sum\limits_{i = 0}^{T} V_{i,j} = U * \frac {\Delta Charge_j}{\Delta t * a * P}$

#### Ré-évaluation des contraintes

Les contraintes du problème deviennent alors
  - Puissance maximale autorisée $P$ : $\forall i \in [0,T],  \sum \limits_{j=1}^N v_{i,j} \leq 1$
  - Heure d'arrivée et de départ des voitures. $\forall j \in [1,N], v_{{i,j}_{i < j_0}} = 0 = v_{{i,j}_{i > j_1}}$
  - Charge obligatoire : $\forall j \in [1,N], \sum\limits_{i = 0}^{T} V_{i,j} = \Delta Charge_j * \frac {U}{\Delta t * a * P} = \Delta Charge_j * K$ avec $K = \frac {U}{\Delta t * a * P}$ constant. 

### Problème final

Finalement, nous en arrivons au problème final, comportant des contraintes individuelles (la charge) et des contraintes couplées (la puissance maximale à ne pas dépasser), tout en prenant en compte le coût des charges:

$$ \min\limits_{\forall i \in [0,T],  \sum \limits_{j=1}^N v_{i,j} \leq 1, \forall j \in [1,N] , \sum\limits_{i = j_0}^{j_1} V_{i,j} = \Delta Charge_j * K} (f(V))$$

# Etude du problème

## Cas d'un unique véhicule

L'étude d'un cas simple tel que le problème à un véhicule est utile afin de comprendre les spécificités de ce problème. Nous allons tout d'abord étudier la fonction objectif, puis l'existence puis l'unicité des solutions de ce problème.

Pour ce cas particulier, on considérera que l'on se place sur l'intervalle de temps correspondant exactement à celui de la charge de la voiture. Cela évite de contrainte la nullité de certaines $v_i$

### Etude de la fonction objectif

On a, pour $V \in \mathbb R^T, f(V) = C*V$

Donc f est linéaire, donc convexe (mais pas fortement).

### Existence et unicité des solutions

Ce problème admettra une solution lorsque les contraintes définissent un domaine non vide. En effet ces contraintes donneraient alors un espace fermé explorable par la fonction objectif, qui est linéaire en dimension finie donc continue.

Le théorème de Weierstrass assure alors l'existence d'une solution au problème de minimisation.

Dans le cas d'un unique véhicule, on a N = 1, ainsi les contraintes se résument à :
 - $\forall i \in [1,T], V_{i,1} \leq 1$
 - $\sum\limits_{i = 1_0}^{1_1} V_{i,1} = \Delta Charge_1 * K$
 
 Le problème est donc solvable si et seulement si lorsque l'on maximise la puissance donnée à la voiture on dépasse la contrainte de charge, donc si et seulement si :
 $$ \frac{1_1 - 1_0}{K} \ge \Delta Charge_1 $$

Nous cherchons aussi une condition nécessaire et suffisante afin d'assurer l'unicité d'une solution. On raisonne donc par disjonction de cas. On se place du point de vue d'une solution au problème.

Deux cas de figure se présentent alors:
 - Si aucune charge n'est réalisée sur une heure pleine
 - Si une partie de la charge est réalisée sur une heure creuse
 
Dans le premier cas: 
 - Si la puissance est maximale pendant toute la durée des heures creuses : La solution est unique.
 - Si non, alors on dispose de deux indices de temps en heures creuses pour lesquels la puissance délivrée à la voiture est différente, on peut alors intervertir les puissances de ces deux indices. On a donc le même coût en électricité, mais pas le même profil de charge : La solution n'est pas unique

Dans le second cas, on a nécessairement une puissance maximale sur les heures creuses, car sinon le coût de charge n'est pas optimal. On a également une disjonction de cas semblable au premier cas:
 - Si la puissance est maximale pendant toute la durée des heures pleines : La solution est unique
 - Si non, alors on dispose de deux indices de temps en heures pleines pour lesquels la puissance délivrée à la voiture est différente, on peut alors intervertir les puissances de ces deux indices. On a donc le même coût en électricité, mais pas le même profil de charge : La solution n'est pas unique

Finalement, on a unicité de la solution si et seulement si on a :
$$ \sum\limits_{i \in heures creuses} 1 = \Delta Charge_1 * K $$ 

ou 

$$ 1_1 - 1_0 = \Delta Charge_1 * K$$

Ce qui en pratique n'arrivera pas.

### Résolution du problème

Dans ce cas simple, nous avons occulté les contraintes de nullité sur les $v_i$, comme dit précédemment.

On peut résoudre ce problème par l'algorithme d'Uzawa

Pour les constantes du problème, nous avons adopté les valeurs suivantes :
 - Puissance totale du centre : $P = 2000 W$
 - Tension de charge unitaire : $U = 400 V$
 - Pente de charge : $a = 0.00184507 A^{-1}$ 

Ceci nous donne un coefficient totale $K = 1083.9697$

Enfin nous avons discrétisé le temps en 10 points, chacun représentant une seconde.

### Résultats

L'algorithme a bien fonctionné, voilà le profil de charge que l'on obtient :